<center><img src="https://github.com/Estadistica-AnalisisPolitico/operations_onDFs/blob/main/Logo2025.png?raw=true" width="900"/></center>

<br>
<br>

Profesor:[Dr. José Manuel MAGALLANES REYES, Ph.D](http://www.pucp.edu.pe/profesor/jose-manuel-magallanes/" target="_blank) <br>

-   Profesor Principal del Departamento de Ciencias Sociales, Sección de Ciencia Política y Gobierno.

-   [Oficina 223](https://goo.gl/maps/xuGeG6o9di1i1y5m6) - Edificio CISEPA / ECONOMIA / CCSS
-   Telefono: (51) 1 - 6262000 anexo 4302
-   Correo Electrónico: [jmagallanes\@pucp.edu.pe](mailto:jmagallanes@pucp.edu.pe)

------------------------------------------------------------------------

In [1]:
klippy::klippy(position = c('top', 'right'))

# Operations on Data Frames

## Aggregation

Let's bring some data of COVID from [Brazil](https://infoms.saude.gov.br/extensions/covid-19_html/covid-19_html.html):

In [2]:
rm(list = ls())
linkCovid='https://github.com/Estadistica-AnalisisPolitico/operations_onDFs/raw/refs/heads/main/data/BrasilCovid.rds'
dataCovid=readRDS(url(linkCovid))

Now, check the data available:

In [3]:
str(dataCovid)

Let's keep complete data by "ESTADO":

In [4]:
dataCovid=dataCovid[dataCovid$estado!="",]

Let's keep some columns:

In [5]:
toSelect=c('regiao', 'estado', 'municipio','data', 'semanaEpi','casosNovos', 'obitosNovos','day','year','month')
covid=dataCovid[,toSelect]

head(covid)

Let's find out about years available:

In [6]:
unique(covid$year)

In [7]:
unique(covid$month)

So, we have data from January to July 2022. Let's find out: **count of new positive cases per month**:

In [8]:
for (month in unique(covid$month)) {
    print(paste(month,sum(covid$casosNovos[covid$month==month])))}

We use **aggregation** to simplify the previous steps:

In [9]:
# sum of cases by month
casesSumByMonth=aggregate(data=covid,casosNovos~month,sum)
casesSumByMonth

Notice the structure:

In [10]:
is.data.frame(casesSumByMonth)

**AGGREGATING** capabilities allow us to produce useful output with few code:

-   **The groupings**:

In the last example, *month* was the **grouping** variable. We can have more than one:

In [11]:
# sum of cases by estado and month
theModel=casosNovos~estado + month
theFun=sum
casesSumByStateAndMonth=aggregate(data=covid,
                                  theModel,
                                  theFun)
casesSumByStateAndMonth

-   **The function to apply**:

We can have more than one function:

In [12]:
# sum and mean of cases by estado and week
theModel=casosNovos~estado + semanaEpi
theFun=function(x) c(mean = mean(x), sum = sum(x) )
casesSumAndMeanByStateAndWeek=aggregate(data=covid,
                                        theModel,
                                        theFun)

# sometimes you can not see this (when in RStudio)
head(casesSumAndMeanByStateAndWeek,30)

If you do not see the above result, turn the aggregation explicitly into a dataframe:

In [13]:
casesSumAndMeanByStateAndWeek=do.call(data.frame,
                                      aggregate(data=covid,
                                                theModel,
                                                theFun))
head(casesSumAndMeanByStateAndWeek,30)

-   **The variables transformed**:

We can apply the function to more than one variable:

In [14]:
# sum of cases and deaths by estado
theModel=cbind(casosNovos,obitosNovos)~estado
theFun=sum
CasesAndDeathsByState=aggregate(data=covid,
                                theModel,
                                theFun)

head(CasesAndDeathsByState,30)

-   Function **according** to variable

The function can vary according to variable. In this case, using **dplyr** is needed:

In [15]:
library(dplyr)
covid |>
  group_by(month) |>
  summarize(casosNovos_VAR = var(casosNovos),
            casosNovos_SD = sd(casosNovos),
            obitosNovos_Median = median(obitosNovos),
            obitosNovos_Mean = mean(obitosNovos))

------------------------------------------------------------------------

## Concatenation

As the name implies, it is the process to unite dataframes.

For this example, there is a webpage in **[fragilestatesindex.org](https://fragilestatesindex.org/excel/)** where we can find several links to excel files. Let me get all the links:

In [16]:
library(rvest)

# Read the HTML content of the website
webpage <- read_html("https://fragilestatesindex.org/excel/")


allLinks=html_nodes(webpage,"table") %>%
            html_nodes("a") %>% #“a” nodes contain linked text.
                html_attr("href")%>% # the url (an html attribute)
                    trimws()%>%unique() # cleaning and keeping non duplicated
allLinks

Now, I will create a list of data frames by loading the data from those links:

In [17]:
dfs = lapply(allLinks, rio::import,col_types = "text")

Can we concatenate the files now?

In [18]:
do.call(rbind, dfs)

It is not possible due to the column names differences in each dataframe (DF).

We can solve this. Let's build a list where each element is the column names of each DF:

In [19]:
allNames=list()
i=1
for (df in dfs){
    allNames[[i]]=names(df)
    i=i+1
}

Let me find WHAT is common in all DFs:

In [20]:
Reduce(intersect, allNames)

If this is the union of all the column names:

In [21]:
Reduce(union, allNames)

This is what is NOT common:

In [22]:
setdiff(Reduce(union, allNames),Reduce(intersect, allNames))

I will force that each DF has the same names and is located in the same position:

In [23]:
allInCommon=Reduce(intersect, allNames)

for (i in 1:length(dfs)){
    dfs[[i]]=dfs[[i]][,allInCommon] #forcing!
}

Now we can concatenate all of them:

In [24]:
allFragility=do.call(rbind, dfs)

Let's check:

In [25]:
str(allFragility)

The concatenation is done, but the result needs cleaning and formatting.

Let's use the column name patterns to create more useful names:

In [26]:
# woking on the names that have a ':'
grep(':',names(allFragility),value = T)

In [27]:
columnNamesToClean=grep(':',names(allFragility),value = T)

# Cleaning: no spaces and to lowercase
gsub(":\\s|\\s","_",columnNamesToClean)%>%tolower()

Let's make the replacement:

In [28]:
# sorted new names
newNames=gsub(":\\s|\\s","_",columnNamesToClean)%>%
            tolower()

# then
names(allFragility)=c(names(allFragility)[1:4],newNames)
str(allFragility)

We can have a better ordering in the column names:

In [29]:
allFragility=cbind(allFragility[,c(1:4)],allFragility[,sort(names(allFragility)[-c(1:4)])])

Now:

In [30]:
head(allFragility)

Let's check some possible errors knowing the origin was an excel file:

- The year:

In [31]:
table(allFragility$Year)

The numeric format tells us the representation of a date:

In [32]:
as.Date(38718, origin="1899-12-30")

This looks like an easy fix, just verify we have integers:

In [33]:
str(allFragility$Year)

Then,

In [34]:
allFragility$Year=as.numeric(allFragility$Year)

We have two years well written, then:

In [35]:
# temporal Column:
allFragility$Year2=format(as.Date(allFragility$Year,
                                  origin="1899-12-30"),
                          "%Y") # I will just recover the year.

# improved column
allFragility$Year=ifelse(allFragility$Year%in%c(2021,2023),
                         allFragility$Year,
                         allFragility$Year2)

# deleting temporal
allFragility$Year2=NULL

# as numeric
allFragility$Year=as.numeric(allFragility$Year)
# now
table(allFragility$Year)

Recheck data types:

In [36]:
str(allFragility)

- Formatting the numeric columns:

In [37]:
allFragility[,-c(1:3)]=lapply(allFragility[,-c(1:3)],as.numeric)

Verifying:

In [38]:
str(allFragility)

We have concatenated, cleaned and formatted successfully

In [39]:
saveRDS(allFragility,'allFragility_ok.rds')

------------------------------------------------------------------------

## Reshaping

Let me recover that data:

In [40]:
linkFragile='https://github.com/Estadistica-AnalisisPolitico/operations_onDFs/raw/refs/heads/main/data/allFragility_ok.rds'
fragile=readRDS(url(linkFragile))

A basic look:

In [41]:
str(fragile)

We have data about countries (index of fragility and its components). The data is from several years:

In [42]:
table(fragile$Year)

Take a look:

In [43]:
head(fragile)

What **shape** does the table have? The presence of year in a column could make us think it is in a long shape.

This could be a LONG:

In [44]:
head(fragile[,c('Country','Year','Total')])

And you also have WIDE:

In [45]:
head(fragile[fragile$Year==2023,c('Country','c1_security_apparatus',	'c2_factionalized_elites',	'c3_group_grievance')])

## From Long to Wide

In [46]:
fragileWide=tidyr::pivot_wider(fragile[,c('Country','Year','Total')],
                               names_from = Year,
                               values_from = Total,
                               names_sort=T)
fragileWide

Notice that for the column to be sorted properly you need to adding *names_sort*.

The wide format is useful in several cases. In general, it looks easy.

In **base R**, you can use it directly for plotting:

In [47]:
boxplot(fragileWide[,-1])

BUT in others such as GGPLOT, it is troublesome to use that format:

In [48]:
library(ggplot2)
base=ggplot(fragileWide)
base+geom_boxplot(aes(x=as.ordered(2006),y=`2006`)) +
     geom_boxplot(aes(x=as.ordered(2010),y=`2010`)) +
     geom_boxplot(aes(x=as.ordered(2015),y=`2015`)) + labs(y='')

## From Wide to Long

We have  **pivot_longer** :

In [49]:
fragileLong=tidyr::pivot_longer(fragileWide,!Country, names_to = "Year", values_to = "FragilityIndex")
fragileLong

GGPLOT will work with this:

In [50]:
base = ggplot(data=fragileLong)
base + geom_boxplot(aes(x=Year,y=FragilityIndex))

We can also use **base R**:

In [51]:
boxplot(data=fragileLong,FragilityIndex~Year)

This is another example without years.

Let me keep one year, and some wide-shaped columns:

In [52]:
CVars_columns=c('c1_security_apparatus',	'c2_factionalized_elites',	'c3_group_grievance')

#only one year
fragile_CVars_wide=fragile[fragile$Year==2020,c('Country',CVars_columns)]

fragile_CVars_wide

In [53]:
boxplot(fragile_CVars_wide[,-1])

In [54]:
fragile_CVars_long=tidyr::pivot_longer(fragile_CVars_wide,!Country, names_to = "CVars_name", values_to = "CVars_value")
fragile_CVars_long

In [55]:
base=ggplot(data=fragile_CVars_long)
base+geom_boxplot(aes(x=CVars_name,y=CVars_value))

## LONG and WIDE madness

Let me keep both long and wide elements:

In [56]:
fragile_since2020=fragile[fragile$Year>=2020,c('Country','Year',CVars_columns)]
fragile_since2020

See this result when sending a long column into a wide format when other variables are already wide:

In [57]:
fragile_since2020_wide=tidyr::pivot_wider(fragile_since2020,names_from = Year, values_from = all_of(CVars_columns))
fragile_since2020_wide

As you saw, the conversion into the wide format changed the column names.

Now, sending the wide columns next to the long column:

In [58]:
fragile_since2020_long=tidyr::pivot_longer(fragile_since2020,
                                           !c(Country,Year),
                                           names_to = "CVars_name",
                                           values_to = "CVars_value")
fragile_since2020_long